# DR missing files
- check sessions one-by-one
- mark each session as checked when finished


Problem sessions:
- `_20220927` (timestamps file needs recovering)
- `_20220825` (half-recording)

In [1]:
import datetime
import pathlib
import shutil
import time
import pprint
from typing import Generator, Optional
import IPython.display

from np_session import Session, sessions as get_sessions

while not (USERNAME := input('Enter username:')):
    continue

problem_sessions = ('_20220825', '_20220927')

def sessions(skip_already_checked=True) -> Generator[Session, None, None]:
    for session in get_sessions('DR'):
        if any(_ in str(session) for _ in problem_sessions):
            continue
        if skip_already_checked and session.state.get('checked'):
            continue
        yield session

def get_files_created_between(
    path: str | pathlib.Path,
    glob: str = "*",
    start: float | datetime.datetime = 0,
    end: Optional[float | datetime.datetime] = None,
) -> list[pathlib.Path]:
    path = pathlib.Path(path)
    if not path.is_dir():
        path = path.parent
    if not end:
        end = time.time()
    start = start.timestamp() if isinstance(start, datetime.datetime) else start
    end = end.timestamp() if isinstance(end, datetime.datetime) else end
    ctime = lambda x: x.stat().st_ctime
    files = (file for file in path.glob(glob) if int(start) <= ctime(file) <= end)
    return sorted(files, key=ctime)

DR_sessions: Generator[Session, None, None] = sessions()

# Get next DR session:

In [2]:
session = next(DR_sessions)

# Display files missing on np-exp:
can be re-run if files have changed 

In [3]:
pprint.pprint(missing := session.get_missing_files())

('*_probeABC',
 '*_probeDEF',
 '*_probeA*_sorted/continuous/Neuropix-PXI-???.1/continuous.dat',
 '*_probeB*_sorted/continuous/Neuropix-PXI-???.1/continuous.dat',
 '*_probeC*_sorted/continuous/Neuropix-PXI-???.1/continuous.dat',
 '*_probeD*_sorted/continuous/Neuropix-PXI-???.1/continuous.dat',
 '*_probeE*_sorted/continuous/Neuropix-PXI-???.1/continuous.dat',
 '*_probeF*_sorted/continuous/Neuropix-PXI-???.1/continuous.dat')


# Platform json:

In [3]:
IPython.display.JSON(session.platform_json.json())

c:\Users\ben.hardcastle\github\np_workflows\.venv\Lib\site-packages\IPython\core\display.py:618: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

# Original files on rig:

In [11]:
print(session.rig)
rig_paths = session.rig.paths
rig_paths.pop('ScriptCamstim', None)
rig_paths.pop('Cam3d', None)
rig_paths.pop('MVR', None)
original_files = dict.fromkeys(rig_paths.keys(), [])
globs = {'VideoMVR': '**/*'}
for program, data_path in rig_paths.items():
    original_files[program] = get_files_created_between(
        path=data_path,
        glob=globs.get(program, '*'), 
        start=session.experiment_start,
        end=session.experiment_end,
        )
pprint.pprint(original_files)

{'Camstim': [WindowsPath('//W10DTSM118294/c$/ProgramData/AIBS_MPE/camstim/data/behavior-params-1653427584.json'),
             WindowsPath('//W10DTSM118294/c$/ProgramData/AIBS_MPE/camstim/data/behavior-init-1653427584.bat'),
             WindowsPath('//W10DTSM118294/c$/ProgramData/AIBS_MPE/camstim/data/mapping-params-1653427584.json'),
             WindowsPath('//W10DTSM118294/c$/ProgramData/AIBS_MPE/camstim/data/opto-params-1653427584.json'),
             WindowsPath('//W10DTSM118294/c$/ProgramData/AIBS_MPE/camstim/data/behavior-3fb9da761fcc479bb781a866340a3aa4-1653427584.pkl'),
             WindowsPath('//W10DTSM118294/c$/ProgramData/AIBS_MPE/camstim/data/220524153159.pkl'),
             WindowsPath('//W10DTSM118294/c$/ProgramData/AIBS_MPE/camstim/data/mapping-init-1653431541.bat'),
             WindowsPath('//W10DTSM118294/c$/ProgramData/AIBS_MPE/camstim/data/220524155752-mapping_script_v2.pkl'),
             WindowsPath('//W10DTSM118294/c$/ProgramData/AIBS_MPE/camstim/data/opto-ini

---
---
# Mark session as checked when np-exp folder is as complete as possible:

Acceptable missing files:
- raw data already in lims:
    - `*_probeABC`
    - `*_probeDEF` 
- large sorted files already deleted:
    - `*_sorted/ ... /continuous.dat`
- surface images

In [ ]:
session.state['missing'] = missing
session.state['checked'] = time.time()
session.state['checked_by'] = USERNAME 